# Лабораторная работа 3. Квантовые измерения.

Предпосылка

- [Глава 1.4 Одиночные кубитные вентили](https://qiskit.org/textbook/ch-states/single-qubit-gates.html)
- [Глава 2.2 Множественные кубиты и запутанные состояния](https://qiskit.org/textbook/ch-gates/multiple-qubits-entangled-states.html)
- [Уменьшение шума на реальных квантовых компьютерах](https://www.youtube.com/watch?v=yuDxHJOKsVA&list=PLOFEBzvs-Vvp2xg9-POLJhQwtVktlYGbY&index=8)

Другие соответствующие материалы

- [Фейнмановские лекции гл. III - 12](https://www.feynmanlectures.caltech.edu/III_12.html)
- [Квантовая операция](https://qiskit.org/documentation/tutorials/circuits/3_summary_of_quantum_operations.html)
- [Интерактивная сфера Блоха](https://nonhermitian.org/kaleido/stubs/kaleidoscope.interactive.bloch_sphere.html#kaleidoscope.interactive.bloch_sphere)
- [Глава 5.2 Уменьшение ошибок измерения](https://qiskit.org/textbook/ch-quantum-hardware/measurement-error-mitigation.html)

In [1]:
from qiskit import *
import numpy as np
from numpy import linalg as la
from qiskit.tools.monitor import job_monitor
import qiskit.tools.jupyter

<h2 style="font-size:24px;">Часть 1: Измерение состояния кубита</h2>

<br>
<div style="background: #E8E7EB; border-radius: 5px;
-moz-border-radius: 5px;">
  <p style="background: #800080;
            border-radius: 5px 5px 0px 0px;
            padding: 10px 0px 10px 10px;
            font-size:18px;
            color:white;
            "><b>Цель</b></p>
    <p style=" padding: 0px 0px 10px 10px;
              font-size:16px;">Определите блоховские компоненты кубита.</p>
</div>

В основе работы квантового компьютера лежит способность вычислять блоховские компоненты кубита или кубитов. Эти компоненты соответствуют средним значениям операторов Паули $X, Y, Z$ и являются важными величинами для таких приложений, как квантовая химия и оптимизация. К сожалению, невозможно одновременно вычислить эти значения, что требует многократного выполнения одной и той же схемы. Кроме того, измерения ограничены вычислительным базисом (Z-базисом), так что каждый Паули должен быть повернут к стандартному базису для доступа к компонентам x и y. Здесь мы проверяем методы, рассматривая случай случайного вектора на сфере Блоха.

<h3 style="font-size: 20px">📓 1. Выразить средние значения операторов Паули для произвольного состояния кубита $|q\rangle$ в вычислительном базисе.</h3>

В качестве примера приведен случай ожидаемого значения Z-ворота Паули. 

Используя диагональное представление, также известное как спектральная форма или ортонормированное разложение, вентиля Паули $Z$ и соотношения между вентилями Паули (см. [здесь](https://qiskit.org/textbook/ch-states/single-qubit-gates.html) ), средние значения вентилей $X, Y, Z$ можно записать как

$$ \begin{align} \langle Z \rangle &amp;=\langle q | Я | q\rangle =\langle q|0\rangle\langle 0|q\rangle - \langle q|1\rangle\langle 1|q\rangle =|\langle 0 |q\rangle|^2 - |\langle 1 | q\rangle|^2\\ \langle X \rangle &amp;= \\ \langle Y \rangle &amp;= \end{aligned} \ $$ соответственно.

Таким образом, математические ожидания Паулиса для состояния кубита $|q\rangle$ могут быть получены путем проведения измерений в стандартном базисе после поворота стандартной базисной системы координат вдоль соответствующей оси. Вероятности получения двух возможных результатов 0 и 1 используются для оценки желаемого ожидаемого значения, как показывают приведенные выше уравнения.

<h3 style="font-size: 20px">2. Измерить координаты кубита на сфере Блоха с помощью симулятора qasm и построить вектор на сфере Блоха.</h3>

<h4 style="font-size: 17px">📓Шаг А. Создайте состояние кубита, используя метод схемы, <code>initialize</code> двумя случайными комплексными числами в качестве параметра.</h4>

Чтобы узнать, как использовать функцию `initialize` , проверьте [здесь](https://qiskit.org/documentation/tutorials/circuits/3_summary_of_quantum_operations.html) . (перейдите в раздел `arbitrary initialization` .)

In [2]:
qc = QuantumCircuit(1)

#### your code goes here




<h4 style="font-size: 17px">📓 Шаг B. Постройте схемы для измерения ожидаемых значений вентиля $X, Y, Z$ на основе ваших ответов на вопрос 1. Запустите ячейку ниже, чтобы оценить координаты сферы Блоха кубита из шага A с помощью симулятора qasm. .</h4>

В качестве примера приведена схема измерения ворот $Z$.

In [3]:
# z measurement of qubit 0
measure_z = QuantumCircuit(1,1)
measure_z.measure(0,0)

# x measurement of qubit 0
measure_x = QuantumCircuit(1,1)
# your code goes here







# y measurement of qubit 0
measure_y = QuantumCircuit(1,1)
# your code goes here







shots = 2**14 # number of samples used for statistics
sim = Aer.get_backend('qasm_simulator')
bloch_vector_measure = []
for measure_circuit in [measure_x, measure_y, measure_z]:
    
    # run the circuit with a the selected measurement and get the number of samples that output each bit value
    counts = execute(qc+measure_circuit, sim, shots=shots).result().get_counts()

    # calculate the probabilities for each bit value
    probs = {}
    for output in ['0','1']:
        if output in counts:
            probs[output] = counts[output]/shots
        else:
            probs[output] = 0
            
    bloch_vector_measure.append( probs['0'] -  probs['1'] )

# normalizing the bloch sphere vector
bloch_vector = bloch_vector_measure/la.norm(bloch_vector_measure)

print('The bloch sphere coordinates are [{0:4.3f}, {1:4.3f}, {2:4.3f}]'
      .format(*bloch_vector))    

<h4 style="font-size: 17px">Шаг C. Постройте вектор на сфере Блоха.</h4>

Обратите внимание, что следующая ячейка для интерактивной bloch_sphere не будет работать должным образом, если вы не работаете в [IQX](https://quantum-computing.ibm.com/login) . Вы можете либо использовать `plot_bloch_vector` для неинтерактивной версии, либо установить `kaleidoscope` , запустив

```
pip install kaleidoscope

```

в терминале. Вам также необходимо перезапустить ядро после установки. Чтобы узнать больше о том, как использовать интерактивную сферу Блоха, перейдите [сюда](https://nonhermitian.org/kaleido/stubs/kaleidoscope.interactive.bloch_sphere.html#kaleidoscope.interactive.bloch_sphere) .

In [4]:
from kaleidoscope.interactive import bloch_sphere

bloch_sphere(bloch_vector, vectors_annotation=True)

In [5]:
from qiskit.visualization import plot_bloch_vector

plot_bloch_vector( bloch_vector )

<h2 style="font-size:24px;">Часть 2: Измерение энергии</h2>

<br>
<div style="background: #E8E7EB; border-radius: 5px;
-moz-border-radius: 5px;">
  <p style="background: #800080;
            border-radius: 5px 5px 0px 0px;
            padding: 10px 0px 10px 10px;
            font-size:18px;
            color:white;
            "><b>Цель</b></p>
    <p style=" padding: 0px 0px 10px 10px;
              font-size:16px;">Оцените энергетические уровни основного состояния водорода с помощью симулятора qasm.</p>
</div>

Энергию квантовой системы можно оценить, измерив среднее значение ее гамильтониана, который является эрмитовым оператором, с помощью процедуры, которую мы освоили в части 1.

Основное состояние водорода не определяется как единственное уникальное состояние, а фактически содержит четыре различных состояния из-за спинов электрона и протона. Во второй части этой лабораторной работы мы оцениваем разницу энергий между этими четырьмя состояниями, полученную из `hyperfine splitting` , вычисляя среднее значение энергии для системы двух спинов с гамильтонианом, выраженным в операторах Паули. Подробнее о `hyperfine structure` см. [здесь](https://www.feynmanlectures.caltech.edu/III_12.html)

Рассмотрим систему с гамильтонианом взаимодействия двух кубитов $H = A(XX+YY+ZZ)$, где $A = 1,47e^{-6} эВ$, а $X, Y, Z$ — вентили Паули. Затем математическое ожидание энергии системы можно оценить, объединив математическое ожидание каждого члена гамильтониана. В этом случае $E = \langle H\rangle = A( \langle XX\rangle + \langle YY\rangle + \langle ZZ\rangle )$. 

<h3 style="font-size: 20px">📓 1. Выразите математическое ожидание каждого члена гамильтониана для произвольного двухкубитного состояния $|\psi \rangle$ в вычислительном базисе.</h3>

Случай с термином $\langle ZZ\rangle$ приведен в качестве примера.

$$ \begin{align} \langle ZZ\rangle &amp;=\langle \psi | ЗЗ | \psi\rangle =\langle \psi|(|0\rangle\langle 0| - |1\rangle\langle 1|)\otimes(|0\rangle\langle 0| - |1\rangle\langle 1|) |\psi\rangle =|\langle 00|\psi\rangle|^2 - |\langle 01 | \psi\rangle|^2 - |\langle 10 | \psi\rangle|^2 + |\langle 11|\psi\rangle|^2\\ \langle XX\rangle &amp;= \\ \langle YY\rangle &amp;= \end{aligned} $$

<h3 style="font-size: 20px">2. Измерить ожидаемую энергию системы с помощью симулятора qasm, когда два кубита запутаны. Рассмотрим основу колокола, четыре различных запутанных состояния.</h3>

<h4 style="font-size: 17px">📓Шаг A. Постройте схемы, чтобы подготовить четыре разных состояния звонка.</h4>

Обозначим каждое состояние звонка как $$ \begin{aligned} Tri1 &amp;= \frac{1}{\sqrt2} (|00\rangle + |11\rangle)\ Tri2 &amp;= \frac{1}{\sqrt2} (|00\rangle - |11\rangle)\ Tri3 &amp;= \frac{1}{\sqrt2} (|01\rangle + |10\rangle)\ Sing &amp;= \frac{1}{\sqrt2} (| 10\rangle - |01\rangle) \end{выровнено} $$

In [6]:
# circuit for the state Tri1
Tri1 = QuantumCircuit(2)
# your code goes here






# circuit for the state Tri2
Tri2 = QuantumCircuit(2)
# your code goes here





# circuit for the state Tri3
Tri3 = QuantumCircuit(2)
# your code goes here






# circuit for the state Sing
Sing = QuantumCircuit(2)
# your code goes here







<h4 style="font-size: 17px">📓Шаг B. Создайте схемы для измерения ожидаемого значения каждого члена гамильтониана на основе вашего ответа на вопрос 1.</h4>

In [135]:
# <ZZ> 
measure_ZZ = QuantumCircuit(2)
measure_ZZ.measure_all()

# <XX>
measure_XX = QuantumCircuit(2)
# your code goes here





# <YY>
measure_YY = QuantumCircuit(2)
# your code goes here






 <h4 style="font-size: 17px">Шаг C. Выполните схемы на симуляторе qasm, запустив ячейку ниже, и оцените ожидаемое значение энергии для каждого состояния.</h4>


In [136]:
shots = 2**14 # number of samples used for statistics

A = 1.47e-6 #unit of A is eV
E_sim = []
for state_init in [Tri1,Tri2,Tri3,Sing]:
    Energy_meas = []
    for measure_circuit in [measure_XX, measure_YY, measure_ZZ]:
    
        # run the circuit with a the selected measurement and get the number of samples that output each bit value
        qc = state_init+measure_circuit
        counts = execute(qc, sim, shots=shots).result().get_counts()

        # calculate the probabilities for each computational basis
        probs = {}
        for output in ['00','01', '10', '11']:
            if output in counts:
                probs[output] = counts[output]/shots
            else:
                probs[output] = 0
            
        Energy_meas.append( probs['00'] - probs['01'] - probs['10'] + probs['11'] )
 
    E_sim.append(A * np.sum(np.array(Energy_meas)))

In [7]:
# Run this cell to print out your results

print('Energy expection value of the state Tri1 : {:.3e} eV'.format(E_sim[0]))
print('Energy expection value of the state Tri2 : {:.3e} eV'.format(E_sim[1]))
print('Energy expection value of the state Tri3 : {:.3e} eV'.format(E_sim[2]))
print('Energy expection value of the state Sing : {:.3e} eV'.format(E_sim[3]))

 <h4 style="font-size: 17px">Шаг D. Понимание результата.</h4>


Если бы вы успешно нашли средние значения энергии, вы бы получили точно такое же значение, $A (= 1,47e^{-6} эВ)$, для тройных значений, $|Tri1\rangle, |Tri2\rangle, | Tri3\rangle$ и один нижний энергетический уровень $-3A (= -4,41e^{-6} эВ)$ для синглетного состояния $|Sing\rangle$.

Здесь мы измерили энергии четырех различных спиновых состояний, соответствующих основному состоянию водорода, и наблюдаемую `hyperfine structure` на энергетических уровнях, вызванную спин-спиновым взаимодействием. Эта крошечная разница в энергии между синглетным и триплетным состояниями является причиной знаменитого излучения с длиной волны 21 см, используемого для картирования структуры галактики. 

В ячейке ниже мы изменяем длину волны излучения от перехода между триплетными состояниями и синглетным состоянием. 

In [8]:
# reduced plank constant in (eV) and the speed of light(cgs units)
hbar, c = 4.1357e-15, 3e10

# energy difference between the triplets and singlet
E_del = abs(E_sim[0] - E_sim[3])

# frequency associated with the energy difference
f = E_del/hbar

# convert frequency to wavelength in (cm) 
wavelength = c/f

print('The wavelength of the radiation from the transition\
 in the hyperfine structure is : {:.1f} cm'.format(wavelength))

<h2 style="font-size:24px;">Часть 3. Выполнение схем на квантовом компьютере</h2>

<br>
<div style="background: #E8E7EB; border-radius: 5px;
-moz-border-radius: 5px;">
  <p style="background: #800080;
            border-radius: 5px 5px 0px 0px;
            padding: 10px 0px 10px 10px;
            font-size:18px;
            color:white;
            "><b>Цель</b></p>
    <p style=" padding: 0px 0px 10px 10px;
              font-size:16px;">Перезапустите схемы на квантовой системе IBM. Выполните уменьшение ошибки измерения в результате, чтобы повысить точность оценки энергии.</p>
</div>

<h4 style="font-size: 17px">Шаг A. Запустите следующие ячейки, чтобы загрузить свою учетную запись и выбрать серверную часть.</h4>

In [139]:
provider = IBMQ.load_account()

In [178]:
backend = provider.get_backend('ibmq_athens')

<h4 style="font-size: 17px">Шаг B. Выполните схемы на квантовой системе.</h4>

В Lab1, когда мы выполнили несколько схем в реальной квантовой системе, мы представили каждую схему как отдельное задание, которое создает несколько идентификаторов заданий. На этот раз мы помещаем все схемы в список и выполняем список цепей как одно задание. Таким образом, выполнение всей схемы может происходить одновременно, что, возможно, сократит время ожидания в очереди.

Кроме того, `transpile` здесь не используется, так как все схемы, которые мы запускаем, состоят из одного или двух вентилей кубита. Мы по-прежнему можем указать начальный_макет и уровень оптимизации через функцию `execute` . Без использования `transpile` транспилированные схемы недоступны, что не является проблемой в данном случае. 

<p>📓 Проверьте информацию о конфигурации бэкенда и карту ошибок через виджет, чтобы определить свой <code>initial_layout</code> .</p>

In [ ]:
# run this cell to get the backend information through the widget
backend

In [ ]:
# assign your choice for the initial layout to the list variable `initial_layout`.
initial_layout = 

Запустите следующую ячейку, чтобы выполнить схемы с initial_layout на серверной части.

In [9]:
qc_all = [state_init+measure_circuit for state_init in [Tri1,Tri2,Tri3,Sing] 
          for measure_circuit in [measure_XX, measure_YY, measure_ZZ] ]  

shots = 8192
job = execute(qc_all, backend, initial_layout=initial_layout, optimization_level=3, shots=shots)
print(job.job_id())
job_monitor(job)

In [159]:
# getting the results of your job
results = job.result()

In [160]:
## To access the results of the completed job
#results = backend.retrieve_job('job_id').result()

<h4 style="font-size: 17px">Шаг C. Оцените уровни энергии основного состояния по результатам предыдущего шага, выполнив ячейки ниже.</h4>

In [161]:
def Energy(results, shots):
    """Compute the energy levels of the hydrogen ground state.
    
    Parameters:
        results (obj): results, results from executing the circuits for measuring a hamiltonian.
        shots (int): shots, number of shots used for the circuit execution.
        
    Returns:
        Energy (list): energy values of the four different hydrogen ground states
    """
    E = []
    A = 1.47e-6

    for ind_state in range(4):
        Energy_meas = []
        for ind_comp in range(3):
            counts = results.get_counts(ind_state*3+ind_comp)
        
            # calculate the probabilities for each computational basis
            probs = {}
            for output in ['00','01', '10', '11']:
                if output in counts:
                    probs[output] = counts[output]/shots
                else:
                    probs[output] = 0
            
            Energy_meas.append( probs['00'] - probs['01'] - probs['10'] + probs['11'] )

        E.append(A * np.sum(np.array(Energy_meas)))
    
    return E

In [10]:
E = Energy(results, shots)

print('Energy expection value of the state Tri1 : {:.3e} eV'.format(E[0]))
print('Energy expection value of the state Tri2 : {:.3e} eV'.format(E[1]))
print('Energy expection value of the state Tri3 : {:.3e} eV'.format(E[2]))
print('Energy expection value of the state Sing : {:.3e} eV'.format(E[3]))

<h4 style="font-size: 17px">Шаг D. Уменьшение ошибки измерения.</h4>

Результаты, полученные при запуске схем в квантовой системе, неточны из-за шума от различных источников, таких как релаксация энергии, расфазировка, перекрестные помехи между кубитами и т. д. На этом этапе мы ослабим влияние шума с помощью уменьшение погрешности измерения. Прежде чем мы начнем, посмотрите это [видео](https://www.youtube.com/watch?v=yuDxHJOKsVA&list=PLOFEBzvs-Vvp2xg9-POLJhQwtVktlYGbY&index=8) . 

In [163]:
from qiskit.ignis.mitigation.measurement import *

<p>📓Создайте схемы для профилирования ошибок измерения всех базовых состояний, используя функцию «complete_meas_cal». Получите объект фильтра измерения, 'meas_filter', который будет применяться к зашумленным результатам, чтобы смягчить ошибку считывания (измерения).</p> 

Для получения дополнительной полезной информации для выполнения этой задачи проверьте [здесь](https://qiskit.org/textbook/ch-quantum-hardware/measurement-error-mitigation.html) . 

In [11]:
# your code to create the circuits, meas_calibs, goes here
meas_calibs, state_labels = 



# execute meas_calibs on your choice of the backend
job = execute(meas_calibs, backend, shots = shots)
print(job.job_id())
job_monitor(job)
cal_results = job.result()
## To access the results of the completed job
#cal_results = backend.retrieve_job('job_id').result()


# your code to obtain the measurement filter object, 'meas_filter', goes here






In [165]:
results_new = meas_filter.apply(results)

In [13]:
E_new = Energy(results_new, shots)

print('Energy expection value of the state Tri1 : {:.3e} eV'.format(E_new[0]))
print('Energy expection value of the state Tri2 : {:.3e} eV'.format(E_new[1]))
print('Energy expection value of the state Tri3 : {:.3e} eV'.format(E_new[2]))
print('Energy expection value of the state Sing : {:.3e} eV'.format(E_new[3]))

<h4 style="font-size: 17px">Шаг E. Интерпретируйте результат.</h4>

<p>📓 Вычислите относительные ошибки (или дробную ошибку) значений энергии для всех четырех состояний с уменьшением ошибки измерения и без него.</p>

In [167]:
# results for the energy estimation from the simulation, 
# execution on a quantum system without error mitigation and
# with error mitigation in numpy array format 
Energy_exact, Energy_exp_orig, Energy_exp_new = np.array(E_sim), np.array(E), np.array(E_new)

In [179]:
# Calculate the relative errors of the energy values without error mitigation 
# and assign to the numpy array variable `Err_rel_orig` of size 4
Err_rel_orig = 

In [169]:
# Calculate the relative errors of the energy values with error mitigation 
# and assign to the numpy array variable `Err_rel_new` of size 4
Err_rel_new = 

In [14]:
np.set_printoptions(precision=3)

print('The relative errors of the energy values for four bell basis\
 without measurement error mitigation : {}'.format(Err_rel_orig))

In [15]:
np.set_printoptions(precision=3)

print('The relative errors of the energy values for four bell basis\
 with measurement error mitigation : {}'.format(Err_rel_new))

<p>📓 Сравните размер ошибок до и после устранения ошибок измерения и обсудите влияние ошибки считывания на информацию карты ошибок выбранного вами бэкэнда.</p> 

**Ваш ответ:**